In [ ]:
from utils.get_data import Get_Opendata_hana
from utils.tools import fiilna
from utils.hana_connection import Upload_Test_Environment,Upload_Deployment_Environment
from input_preprocess.data_preprocessing import Openso_Preprocess, Upload_Data,merge_open_so_wo
from input_preprocess.Indicator_Construction import Foward_Indicator, Trend_Table
from dateutil.relativedelta import relativedelta
import pandas as pd
import datetime

In [ ]:
Open_so_final=Get_Opendata_hana()

d:\SCM_indicator\utils\get_data.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  open_so_ori=pd.read_sql(sql_schema_open_so, extract_connection)
d:\SCM_indicator\utils\get_data.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  open_so_c_ori=pd.read_sql(sql_schema_open_so_c, extract_connection)


In [3]:
today=datetime.datetime.today()
start_date=(today+relativedelta(months=-6)).strftime('%Y-%m-01')
end_date=(today+relativedelta(months=2)).strftime('%Y-%m-01') #要預測的月
full_date_range = pd.date_range(start=start_date,end=end_date,freq='MS').normalize() 
print('現在時間(年/月):',today.strftime('%Y-%m-01'))
print('預測時間(年/月):',end_date)
print(f'資料時間(年/月): {start_date} to {end_date}')

現在時間(年/月): 2024-11-01
預測時間(年/月): 2025-01-01
資料時間(年/月): 2024-05-01 to 2025-01-01


In [4]:
file_path='./Dataset/Open_data_'+today.strftime('%Y-%m-%d')+'.csv'
Open_so_final.to_csv(file_path, index=False)
file_path

'./Dataset/Open_data_2024-11-04.csv'

In [ ]:
MSU=False
def Main_RBU(ship_plant):
    
    Region_class=ship_plant # CN:China (CKH是HUB廠), TW:Taiwan, EU , US
    ABC_class=('A','B','C') # A or B or C ABC_INDICATOR.str.match("^[ABC]")
    PG_class=['ISG','IAG','ICWG','ESG','ECG', 'IPSG','IDSG'] #IIOT->4['ISG','IAG','ICWG','ESG'] EIOT->3 ['ECG', 'IPSG','IDSG']
    #根據上述條件建立sql
    Region_condition = "SHIP_PLANT.str.startswith(@Region_class)"
    ABC_condition = "ABC_INDICATOR.str.startswith(@ABC_class)"
    PG_condition = "PG in @PG_class"
    date_condition = "year_month >= @start_date and year_month <= @end_date"

    if MSU:
        Constrain_query=f"{ABC_condition} and {PG_condition} and {date_condition}"
    else:
        Constrain_query=f"{ABC_condition} and {PG_condition} and {Region_condition} and {date_condition}"
  
    Openso_1=Openso_Preprocess(data=Open_so_final,version_year=1).query(Constrain_query).reset_index(drop=True)
    Openso_2=Openso_Preprocess(data=Open_so_final,version_year=2).query(Constrain_query).reset_index(drop=True)
    Openso_3=Openso_Preprocess(data=Open_so_final,version_year=3).query(Constrain_query).reset_index(drop=True)

    
    # 料號日期條件
    Material_sql=' PART_NO==@PART_NO_name '
    date_column='year_month'
    new_data=pd.DataFrame()
    today_YM=today.strftime('%Y-%m-01')

    for PART_NO_name in Openso_1.PART_NO.unique():
        #open part
        PART_NO_Openso_1=Openso_1.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
        PART_NO_Openso_2=Openso_2.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
        PART_NO_Openso_3=Openso_3.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
        '''
        填補so & openso
        '''
        PART_NO_Openso_1=fiilna(PART_NO_Openso_1, full_date_range , QTY_name= 'OPEN_QTY' )
        PART_NO_Openso_2=fiilna(PART_NO_Openso_2, full_date_range , QTY_name= 'OPEN_QTY' )
        PART_NO_Openso_3=fiilna(PART_NO_Openso_3, full_date_range , QTY_name= 'OPEN_QTY' )

        #計算兩種backlog變化率的計算
        Indicator= Foward_Indicator(PART_NO_Openso_1, PART_NO_Openso_2, PART_NO_Openso_3 )
        #判定條件是否為趨勢
        Trend_Indicator =Trend_Table(Indicator)
        
        temp=pd.DataFrame()
        temp['MATNR']=[PART_NO_name]
        temp['AI_TREND']=Trend_Indicator.loc[Trend_Indicator['Date'] == today_YM, 'labels'].values
        new_data = pd.concat([new_data, temp], ignore_index=True)

    Commit_data= Upload_Data(new_data,Region_class,today)
    #upload測試機/正式機
    # Upload_Test_Environment(Commit_data)
    # Upload_Deployment_Environment(Commit_data)
    print('地區:', Region_class)
    print("Commit data shape:",Commit_data.shape)
    print("上升料號數:",len(Commit_data[Commit_data['AI_TREND']==1]))
    print("下降料號數:",len(Commit_data[Commit_data['AI_TREND']==-1]))
    return Commit_data

In [5]:
Region=['CN','TW','EU','US']
for ship_plant in Region:
    Commit_data= Main_RBU(ship_plant)

地區: CN
Commit data shape: (475, 14)
上升料號數: 16
下降料號數: 29
地區: TW
Commit data shape: (503, 14)
上升料號數: 40
下降料號數: 28
地區: EU
Commit data shape: (436, 14)
上升料號數: 25
下降料號數: 19
地區: US
Commit data shape: (327, 14)
上升料號數: 21
下降料號數: 10


In [6]:
Commit_data

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,AIMB-786G2-00A3,USH1,R,202501,20241129,0,0,0,0,0,0,,0
1,168,IPC-5120-35D,USH1,R,202501,20241129,0,0,0,0,0,0,,0
2,168,PCE-5132G2-00A2,USH1,R,202501,20241129,0,0,0,0,0,0,,0
3,168,ADAM-6250-B,USH1,R,202501,20241129,0,0,0,0,0,0,,-1
4,168,ADAM-5000L/TCP-BE,USH1,R,202501,20241129,0,0,0,0,0,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,168,BB-4WSD9OTB,USH1,R,202501,20241129,0,0,0,0,0,0,,0
323,168,BB-ESW108-A,USH1,R,202501,20241129,0,0,0,0,0,0,,0
324,168,PCL-101100S-1E,USH1,R,202501,20241129,0,0,0,0,0,0,,0
325,168,EKI-2525M-BE,USH1,R,202501,20241129,0,0,0,0,0,0,,0


In [18]:
'''
Finish 測試資料100筆上傳測試機
Finish 上傳正式機
地區: CN
Commit data shape: (471, 14)
上升料號數: 5
下降料號數: 35
Finish 測試資料100筆上傳測試機
Finish 上傳正式機
地區: TW
Commit data shape: (504, 14)
上升料號數: 7
下降料號數: 28
Finish 測試資料100筆上傳測試機
Finish 上傳正式機
地區: EU
Commit data shape: (435, 14)
上升料號數: 9
下降料號數: 17
Finish 測試資料100筆上傳測試機
Finish 上傳正式機
地區: US
Commit data shape: (323, 14)
上升料號數: 5
下降料號數: 11
'''

'\nFinish 測試資料100筆上傳測試機\nFinish 上傳正式機\n地區: CN\nCommit data shape: (471, 14)\n上升料號數: 5\n下降料號數: 35\nFinish 測試資料100筆上傳測試機\nFinish 上傳正式機\n地區: TW\nCommit data shape: (504, 14)\n上升料號數: 7\n下降料號數: 28\nFinish 測試資料100筆上傳測試機\nFinish 上傳正式機\n地區: EU\nCommit data shape: (435, 14)\n上升料號數: 9\n下降料號數: 17\nFinish 測試資料100筆上傳測試機\nFinish 上傳正式機\n地區: US\nCommit data shape: (323, 14)\n上升料號數: 5\n下降料號數: 11\n'

In [ ]:
Region_class='US' # CN:China (CKH是HUB廠), TW:Taiwan, EU , US
ABC_class=('A','B','C') # A or B or C ABC_INDICATOR.str.match("^[ABC]")
PG_class=['ISG','IAG','ICWG','ESG','ECG', 'IPSG','IDSG'] #IIOT->4['ISG','IAG','ICWG','ESG'] EIOT->3 ['ECG', 'IPSG','IDSG']
#根據上述條件建立sql
Constrain_query="ABC_INDICATOR.str.startswith(@ABC_class)  and PG in @PG_class and SHIP_PLANT.str.startswith(@Region_class) and  year_month>=@start_date and year_month<=@end_date"


In [13]:
Openso_1=Openso_Preprocess(data=Open_so_final,version_year=1).query(Constrain_query).reset_index(drop=True)
Openso_2=Openso_Preprocess(data=Open_so_final,version_year=2).query(Constrain_query).reset_index(drop=True)
Openso_3=Openso_Preprocess(data=Open_so_final,version_year=3).query(Constrain_query).reset_index(drop=True)

In [14]:
# 料號日期條件
Material_sql=' PART_NO==@PART_NO_name '
date_column='year_month'
new_data=pd.DataFrame()
today_YM=today.strftime('%Y-%m-01')

for PART_NO_name in Openso_1.PART_NO.unique():
    #open part
    PART_NO_Openso_1=Openso_1.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_2=Openso_2.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_3=Openso_3.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    '''
    填補so & openso
    '''
    PART_NO_Openso_1=fiilna(PART_NO_Openso_1, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_2=fiilna(PART_NO_Openso_2, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_3=fiilna(PART_NO_Openso_3, full_date_range , QTY_name= 'OPEN_QTY' )

    #計算兩種backlog變化率的計算
    Indicator= Foward_Indicator(PART_NO_Openso_1, PART_NO_Openso_2, PART_NO_Openso_3 )
    #判定條件是否為趨勢
    Trend_Indicator =Trend_Table(Indicator)

    temp=pd.DataFrame()
    temp['MATNR']=[PART_NO_name]
    temp['AI_TREND']=Trend_Indicator.loc[Trend_Indicator['Date'] == today_YM, 'labels'].values
    new_data = pd.concat([new_data, temp], ignore_index=True)

Commit_data= Upload_Data(new_data,Region_class,today)
#upload測試機/正式機
# Upload_Test_Environment(Commit_data)
# Upload_Deployment_Environment(Commit_data)
Commit_data

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,PCM-24R2GL-AE,USH1,R,202501,20241104,0,0,0,0,0,0,,1
1,168,SQR-SD4N8G2K6SNBCB,USH1,R,202501,20241104,0,0,0,0,0,0,,0
2,168,IPC-510BP-00XBE,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
3,168,HPC-7442MB-00XE,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
4,168,MIC-75M13-00B1,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,168,BB-SL30500110,USH1,R,202501,20241104,0,0,0,0,0,0,,0
566,168,WISE-2410-EB,USH1,R,202501,20241104,0,0,0,0,0,0,,0
567,168,EKI-1221I-CE,USH1,R,202501,20241104,0,0,0,0,0,0,,0
568,168,989243PF20E,USH1,R,202501,20241104,0,0,0,0,0,0,,0


In [15]:
Commit_data[Commit_data['AI_TREND']==1].reset_index(drop=True)

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,PCM-24R2GL-AE,USH1,R,202501,20241104,0,0,0,0,0,0,,1
1,168,ADAM-6024-D,USH1,R,202501,20241104,0,0,0,0,0,0,,1
2,168,MIC-770H-00A2,USH1,R,202501,20241104,0,0,0,0,0,0,,1
3,168,SQF-SMSM1-16G-SBC,USH1,R,202501,20241104,0,0,0,0,0,0,,1
4,168,IPC-510MB-00XCE,USH1,R,202501,20241104,0,0,0,0,0,0,,1
5,168,UNO-2484G-7731BE,USH1,R,202501,20241104,0,0,0,0,0,0,,1
6,168,PCE-7129G2-00A2E,USH1,R,202501,20241104,0,0,0,0,0,0,,1
7,168,BB-232LPTTL,USH1,R,202501,20241104,0,0,0,0,0,0,,1
8,168,BB-485LDRC9,USH1,R,202501,20241104,0,0,0,0,0,0,,1
9,168,IPC-5122-00B,USH1,R,202501,20241104,0,0,0,0,0,0,,1


In [16]:
Commit_data[Commit_data['AI_TREND']==-1].reset_index(drop=True)

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,IPC-510BP-00XBE,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
1,168,HPC-7442MB-00XE,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
2,168,MIC-75M13-00B1,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
3,168,MIC-770H-20A1,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
4,168,ADAM-6250-B,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,168,AMF-S25V4-512GDSBC,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
121,168,SQF-SHMM2-64G-SBC,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
122,168,ADAM-5069-AE,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
123,168,AIMB-285G2-00A3E,USH1,R,202501,20241104,0,0,0,0,0,0,,-1
